## Setup

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime

### Constants

In [ ]:
PER_PACKET_COLS = ['_time', 'packet_interarrival_time', 'packet_jitter_raw', 'packet_jitter_weighted', 'packet_owd']
AGGREGATE_COLS  = ['_time', 'availability_2ms', 'availability_4ms', 'availability_8ms', 'availability_16ms', 'availability_32ms', 'availability_64ms', 'availability_128ms', 'packet_loss', 'throughput']

In [ ]:
BASE_FILE_PATH = "/home/shared/validation_backups/influx"
BASE_CASE_FILE_PATH = f"{BASE_FILE_PATH}/base_case"
TWO_PCT_LOSS_FILE_PATH = f"{BASE_FILE_PATH}/two_percent_pl"
FIFTEEN_PCT_LOSS_FILE_PATH = f"{BASE_FILE_PATH}/fifteen_percent_pl" 
FIVE_MS_DELAY_FILE_PATH = f"{BASE_FILE_PATH}/five_ms_delay"
FIFTY_MS_DELAY_FILE_PATH = f"{BASE_FILE_PATH}/fifty_ms_delay"
ONE_HUNDRED_MS_DELAY_FILE_PATH = f"{BASE_FILE_PATH}/one_hundred_ms_delay"
MEDIUM_PAYLOAD_FILE_PATH = f"{BASE_FILE_PATH}/436_byte_payload"
LARGE_PAYLOAD_FILE_PATH = f"{BASE_FILE_PATH}/1432_byte_payload"

MINIMAL_BASE_FILES = ["sensor_continuous_low_bw_long_dur_low_pl.csv", "cbr_low_bw_med_dur_low_pl.csv", "cbr_high_bw_low_dur_low_pl.csv"]
MINIMAL_AGGREGATE_FILES = list(map(lambda filename: f"{filename.split('.')[0]}_aggregate.csv", MINIMAL_BASE_FILES))
ALL_BASE_FILES = MINIMAL_BASE_FILES + ["sensor_continuous_low_bw_xlong_dur_low_pl.csv", "cbr_med_bw_med_dur_low_pl.csv"]
ALL_AGGREGATE_FILES = list(map(lambda filename: f"{filename.split('.')[0]}_aggregate.csv", ALL_BASE_FILES))

### Utility functions

In [ ]:
def convert_timestamp_to_datetime(timestamp, initial_datetime):
    """
    Converts a string-timestamp of ISO8601-format to Pandas datetime object,
    normalized to 01. January 1970 
    """
    delta = pd.to_datetime(timestamp, format="ISO8601") - initial_datetime 
    return pd.to_datetime(delta.value)

In [ ]:
def read_per_packet_csv(filepath):
    """
    Reads a per-packet csv, selects only the relevant columns and parses the timestamp to
    a normalized value 
    """
    df = pd.read_csv(filepath, skiprows=3)[PER_PACKET_COLS]
    df["_time"] = df["_time"].apply(lambda ts: convert_timestamp_to_datetime(ts, pd.to_datetime(df["_time"].iloc[0], format="ISO8601")))
    
    return df

def read_aggregate_packet_csv(filepath):
    """
    Reads an aggregate packet csv, selects only the relevant columns and parses the timestamp to
    a normalized value 
    """
    df = pd.read_csv(filepath, skiprows=3)[AGGREGATE_COLS]
    df["_time"] = df["_time"].apply(lambda ts: convert_timestamp_to_datetime(ts, pd.to_datetime(df["_time"].iloc[0], format="ISO8601")))
    
    return df

In [ ]:
def group_by_column(df, col, agg_func, freq):
    return df[["_time", col]].groupby(pd.Grouper(key="_time", freq=freq))\
                            .agg([agg_func])\
                            .dropna()\
                            .droplevel(0, axis=1)\
                            .rename(columns={agg_func: col})\
                            .reset_index()

def group_and_plot_df_by_col(dfs, col, agg_func, freq):
    dfs_aggregated = [group_by_column(df, col, agg_func, freq).rename(columns={col: f"{col}_{index}"}) for index, df in enumerate(dfs)]
    new_df = dfs_aggregated[0]

    for i in range(1, len(dfs_aggregated)):
        new_df = pd.merge(new_df, dfs_aggregated[i], on="_time")

    return px.line(new_df, x="_time", y=list(filter(lambda x: x != "_time", new_df.columns)), markers=True, title=col)

In [ ]:
def plot_aggregate_dfs_by_col(dfs, col):
    new_df = dfs[0][["_time", col]]
    new_df = new_df.rename(columns={col: f"{col}_0"})

    for i in range(1, len(dfs)):
        new_df = pd.merge(new_df, dfs[i][["_time", col]], on="_time").rename(columns={col: f"{col}_{i}"})

    return px.line(new_df, x="_time", y=list(filter(lambda x: x != "_time", new_df.columns)), markers=True, title=col)

## Analysis

### Read per-packet CSVs

In [ ]:
df = read_per_packet_csv(f"{BASE_CASE_FILE_PATH}/{MINIMAL_BASE_FILES[1]}")
df2 = read_per_packet_csv(f"{TWO_PCT_LOSS_FILE_PATH}/{MINIMAL_BASE_FILES[1]}")
df3 = read_per_packet_csv(f"{FIFTY_MS_DELAY_FILE_PATH}/{MINIMAL_BASE_FILES[1]}")
# Rest of DFs comes here

### Plots of per-packet KPI

In [ ]:
dfs = [df, df2, df3]
fig = group_and_plot_df_by_col(dfs, "packet_owd", "mean", "1s")

fig.show()

### Read aggregate CSVs

In [ ]:
df_a = read_aggregate_packet_csv(f"{TWO_PCT_LOSS_FILE_PATH}/{MINIMAL_AGGREGATE_FILES[1]}")
df_a2 = read_aggregate_packet_csv(f"{BASE_CASE_FILE_PATH}/{MINIMAL_AGGREGATE_FILES[1]}")
df_a3 = read_aggregate_packet_csv(f"{FIFTEEN_PCT_LOSS_FILE_PATH}/{MINIMAL_AGGREGATE_FILES[1]}")
df_a5 = read_aggregate_packet_csv(f"{FIFTY_MS_DELAY_FILE_PATH}/{MINIMAL_AGGREGATE_FILES[1]}")
df_a6 = read_aggregate_packet_csv(f"{ONE_HUNDRED_MS_DELAY_FILE_PATH}/{MINIMAL_AGGREGATE_FILES[1]}")

### Plots of aggregate KPIs

In [ ]:
dfs = [df_a, df_a2, df_a3, df_a5, df_a6]
fig = plot_aggregate_dfs_by_col(dfs, "packet_loss")
fig.show()